<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Выбор-модели" data-toc-modified-id="Выбор-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Выбор модели</a></span><ul class="toc-item"><li><span><a href="#Константная-модель" data-toc-modified-id="Константная-модель-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Константная модель</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Градиентный-бустинг" data-toc-modified-id="Градиентный-бустинг-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Градиентный бустинг</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**ТЗ**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.


Постройте модель со значением метрики качества F1 не меньше 0.75. 




## Подготовка

In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import nltk

from catboost import CatBoostClassifier

In [2]:
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv', encoding='utf-8')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.query('toxic == 1').head()

,Unnamed: 0,text,toxic
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,16,"Bye! \n\nDon't look, come or think of comming ...",1
42,42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
# Ненужное поле
df = df.drop('Unnamed: 0', axis=1)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
df.shape

(159292, 2)

Теперь нужно лематизировать твиты и избавиться от лишних символов. Шаги: 
- Создадим столбец с оригинальным текстом
- Напишем lambda функцию для удаления лишних символов и приведения текстов к нижнему регистру
- Удалим слова, не несущие смысловой нагрузки
- Лематизируем текст

In [8]:
df['original_text'] = df['text'].copy()
df = df.reindex(columns=['original_text', 'text', 'toxic'])
df.head()

,original_text,text,toxic
0,Explanation\nWhy the edits made under my usern...,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...","Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...","""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...","You, sir, are my hero. Any chance you remember...",0


In [10]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

tqdm.pandas()
df['lemmatized_text'] = df['text'].progress_apply(lambda x: ' '.
                                         join([lemmatizer.lemmatize(word, get_wordnet_pos(word))\
                                               for word in nltk.word_tokenize(x)]))

100%|██████████| 159292/159292 [24:14<00:00, 109.50it/s]


In [11]:
df.head()

,original_text,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...
2,"Hey man, I'm really not trying to edit war. It...","Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not try to edit war . It..."
3,"""\nMore\nI can't make any real suggestions on ...","""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...
4,"You, sir, are my hero. Any chance you remember...","You, sir, are my hero. Any chance you remember...",0,"You , sir , be my hero . Any chance you rememb..."


In [12]:
df['lemmatized_text'] = df['lemmatized_text'].apply(lambda x: " ".join(re.sub(r'[^a-zA-Z ]', ' ', x).split()))
df.head()

,original_text,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...","Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...","""\nMore\nI can't make any real suggestions on ...",0,More I ca n t make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...","You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


**Предварительно, обработанный текст уберём в отдельный столбец**

**Разделяем данные**

Разделим данные на 3 выборки: обучающую, валидационную и тестовую

In [13]:
features = df['lemmatized_text']
target = df['toxic']

In [14]:
X_train, X, y_train, y = train_test_split(features,
                                          target,
                                          test_size=0.5,
                                          random_state=42)

X_test, X_valid, y_test, y_valid = train_test_split(X,
                                                    y,
                                                    test_size=0.5,
                                                    random_state=42)

print(f'X_train: {X_train.shape}\n' 
      f'X_test: {X_test.shape}\n'
      f'X_valid: {X_valid.shape}\n'
      f'\n'
      f'y_train: {y_train.shape}\n'
      f'y_test: {y_test.shape}\n'
      f'y_valid: {y_valid.shape}')

X_train: (79646,)
X_test: (39823,)
X_valid: (39823,)

y_train: (79646,)
y_test: (39823,)
y_valid: (39823,)


In [15]:
stop_words = set(stopwords.words('english'))

In [37]:
TF_IDF_vec = TfidfVectorizer(
    ngram_range=(1, 1),
    stop_words=stop_words,
    smooth_idf=2, 
    sublinear_tf=2
)

In [38]:
TF_IDF_vec.fit(X_train)

X_train_vectorized = TF_IDF_vec.transform(X_train)
X_test_vectorized = TF_IDF_vec.transform(X_test)
X_valid_vectorized = TF_IDF_vec.transform(X_valid)

In [35]:
print(f'X_train: {X_train_vectorized.shape}\n' 
      f'X_test: {X_test_vectorized.shape}\n'
      f'X_valid: {X_valid_vectorized.shape}')

X_train: (79646, 106079)
X_test: (39823, 106079)
X_valid: (39823, 106079)


## Обучение

### Выбор модели

Так как мы решаем задачу классификации, нужно посмотреть на баланс классов

In [19]:
pos = df.query('toxic == 0').shape
neg = df.query('toxic == 1').shape

difference = (100 / pos[0]) * neg[0]
print(f'{difference} %')

11.310497114027365 %


Позитивные и негативные комментарии соотносятся почти 9:1

Для подбора оптимальной модели дерева и леса напишем класс OptimalModel

Принципы ООП оптимизируют процесс поиска лучших моделей:

- **Абстракция.** Класс абстрагирует концепцию оптимальных моделей от конкретной реализации. Он предоставляет абстрактный интерфейс для работы с моделями
- **Полиморфизм.** В коде используются методы decision_tree и random_forest, которые предоставляют одинаковый интерфейс для работы с разными моделями машинного обучения
- **Инкапсуляция.** Атрибуты X_valid и y_valid и методы decision_tree и random_forest объединены в одном объекте. Данные и методы для работы с ними объединены внутри класса без доступа клиента (внешнего кода)



На выходе мы получим топ 5 моделей

In [20]:
class OptimalModel:
    def __init__(self, X_train, X_valid, y_train, y_valid):
        self.X_train = X_train
        self.X_valid = X_valid
        self.y_train = y_train
        self.y_valid = y_valid
        
    def decision_tree(self):
        metrics = pd.DataFrame(columns=['depth', 'f1_score'])
        
        for i in range(1, 11):
            model = DecisionTreeClassifier(max_depth=i,
                                           random_state=42,
                                           class_weight={0: 1.131, 1: 8.869})
            model.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_valid)
            f1 = f1_score(self.y_valid, y_pred)
            
            metrics = metrics.append({
                'depth': i,
                'f1_score': f1
            }, ignore_index=True)
            
        return metrics.sort_values(by='f1_score', ascending=False).head()
    
    def random_forest(self):
        metrics = pd.DataFrame(columns=['depth', 'estimators', 'f1_score'])
        
        for i in range(1, 11):
            for j in range(1, 11):
                model = RandomForestClassifier(max_depth=i,
                                               n_estimators=j,
                                               random_state=42,
                                               class_weight={0: 1.131, 1: 8.869})
                model.fit(self.X_train, self.y_train)
                y_pred = model.predict(self.X_valid)
                f1 = f1_score(self.y_valid, y_pred)
                
                metrics = metrics.append({
                    'depth': i,
                    'estimators': j,
                    'f1_score': f1
                }, ignore_index=True)
                
        return metrics.sort_values(by='f1_score', ascending=False).head()

In [21]:
optimal_model = OptimalModel(X_train_vectorized,
                             X_valid_vectorized,
                             y_train,
                             y_valid)

#### Константная модель

In [22]:
%%time
const_model = DummyClassifier()
const_model.fit(X_train_vectorized, y_train)
y_pred_const = const_model.predict(X_valid_vectorized)
const_f1 = f1_score(y_valid, y_pred_const)
const_f1

CPU times: user 8.28 ms, sys: 7.31 ms, total: 15.6 ms
Wall time: 14.1 ms


0.0

#### Логистическая регрессия

In [50]:
%%time
model_reg = LogisticRegression(
    random_state=12345,
    solver='liblinear',
    class_weight={0: 1.131, 1: 8.869},
    penalty='l1'
)
model_reg.fit(X_train_vectorized, y_train)
y_pred_reg = model_reg.predict(X_valid_vectorized)
reg_f1 = f1_score(y_valid, y_pred_reg)
reg_f1

CPU times: user 1.32 s, sys: 24.4 ms, total: 1.35 s
Wall time: 1.35 s


0.7585976627712856

#### Дерево решений

In [24]:
%%time
optimal_model.decision_tree()

CPU times: user 21.5 s, sys: 13.6 ms, total: 21.5 s
Wall time: 21.5 s


,depth,f1_score
9,10.0,0.577913
8,9.0,0.562691
7,8.0,0.553375
6,7.0,0.536585
5,6.0,0.535159


#### Случайный лес

In [25]:
%%time
optimal_model.random_forest()

CPU times: user 24.3 s, sys: 37.9 ms, total: 24.3 s
Wall time: 24.3 s


,depth,estimators,f1_score
99,10.0,10.0,0.379930
89,9.0,10.0,0.365927
98,10.0,9.0,0.358213
97,10.0,8.0,0.354497
79,8.0,10.0,0.344011


#### Градиентный бустинг

In [26]:
boost_model = CatBoostClassifier(iterations=60,
                                 random_state=42,
                                 class_weights={0: 1.131, 1: 8.869})
boost_model.set_params(loss_function='Logloss')
boost_model.fit(X_train_vectorized, y_train)
y_pred_boost = boost_model.predict(X_valid_vectorized)
f1_boost = f1_score(y_valid, y_pred_boost)
f1_boost

Learning rate set to 0.5
0:	learn: 0.5590003	total: 2.33s	remaining: 2m 17s
1:	learn: 0.5200435	total: 4.69s	remaining: 2m 16s
2:	learn: 0.4911288	total: 7.03s	remaining: 2m 13s
3:	learn: 0.4633091	total: 9.34s	remaining: 2m 10s
4:	learn: 0.4473536	total: 11.6s	remaining: 2m 8s
5:	learn: 0.4365438	total: 13.9s	remaining: 2m 5s
6:	learn: 0.4245652	total: 16.1s	remaining: 2m 1s
7:	learn: 0.4152895	total: 18.3s	remaining: 1m 58s
8:	learn: 0.4069288	total: 20.6s	remaining: 1m 56s
9:	learn: 0.3984782	total: 22.8s	remaining: 1m 54s
10:	learn: 0.3939282	total: 25s	remaining: 1m 51s
11:	learn: 0.3894658	total: 27.2s	remaining: 1m 48s
12:	learn: 0.3848641	total: 29.5s	remaining: 1m 46s
13:	learn: 0.3813708	total: 31.7s	remaining: 1m 44s
14:	learn: 0.3769525	total: 33.9s	remaining: 1m 41s
15:	learn: 0.3722915	total: 36.2s	remaining: 1m 39s
16:	learn: 0.3693178	total: 38.4s	remaining: 1m 37s
17:	learn: 0.3662761	total: 40.6s	remaining: 1m 34s
18:	learn: 0.3626146	total: 42.8s	remaining: 1m 32s
19

0.7321225879682179

**Вывод**

Из всех моделей нам подошла только логистическая регрессия

### Тестирование модели

In [51]:
%%time
test_model = LogisticRegression(
    random_state=12345,
    solver='liblinear',
    class_weight={0: 1.131, 1: 8.869},
    penalty='l1'
)

test_model.fit(X_train_vectorized, y_train)
y_pred = test_model.predict(X_test_vectorized)
f1 = f1_score(y_test, y_pred)
f1

CPU times: user 1.26 s, sys: 23.9 ms, total: 1.28 s
Wall time: 1.28 s


0.7504059382973788

In [52]:
print(f'Метрика на валидационной выборке: {reg_f1.__round__(3)}\n'
      f'Метрика на тестовой выборке: {f1.__round__(3)}')

Метрика на валидационной выборке: 0.759
Метрика на тестовой выборке: 0.75


**Вывод**

- Мы обучили на получившихся данных 5 типов моделей: константную модель, логистическюу регрессию, дерево решений, случайный лес, градиентный бустинг
- За ключевую метрику была взята F1 мера больше 0.75
- Оптимальной моделью с метрикой больше 0.75 оказалась логистическая регрессия
- На тестовой выборке модель была проверена на переобучаемость: модель не склонна к переобучению

## Выводы

**Общий вывод**

- Были проанализированы и обработаны данные по твитам пользователей
- Данные необходимо было предобработать: удалить лишнее поле, лемматизировать текст и убрать из него лишние символы
- Тексты было необходимо перевести в векторный вид
- Получившиеся данные сформировали датасет для обучения
- Были опробированы 5 алгоритмов машинного обучения
    - DummyClassifier - 0.0
    - LogisticRegression - 0.752
    - DecisionTreeClassifier - 0.562
    - RandomForestClassifier - 0.327
    - CatboostClassifier - 0.727
- Оптимальную метрику демонстрирует модель логистической регрессии
- Итоговая модель не склонна к переобучению: показывает 0.762 на валидационной выборке и 0.753 на тестовой, что выше метрики, указанной в ТЗ